In [22]:
import simtk.openmm.app as app
import simtk.openmm as openmm
import simtk.unit as unit
from pkg_resources import resource_filename
import numpy as np
import os

from openmmtools.constants import kB
from perses.utils.smallmolecules import render_protein_residue_atom_mapping
from perses.rjmc import topology_proposal
from openmmforcefields.generators import SystemGenerator

temperature = 300*unit.kelvin
# Compute kT and inverse temperature.
kT = kB * temperature
beta = 1.0 / kT
ENERGY_THRESHOLD = 1e-6
PROHIBITED_RESIDUES = ['CYS']

In [23]:
def generate_old_top_pos_sys(res_name):
    """
    Generate topology, positions, system to be parametrized with amber14ffsb in vacuum
    """
    pdb = app.PDBFile(f"../../input/{res_name.lower()}_vacuum.pdb")
    topology = pdb.topology
    positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
    
    system_generator = SystemGenerator(['amber14/protein.ff14SB.xml'],
                                   barostat=None,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                        'ewaldErrorTolerance': 1e-4,
                                                        'constraints' : app.HBonds,
                                                        'hydrogenMass' : 4 * unit.amus},
                                    nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                    small_molecule_forcefield='gaff-2.11',
                                    molecules=None,
                                    cache=None)

    system = system_generator.create_system(topology) # Update the parametrization scheme to amberff14sb

    return topology, positions, system, system_generator

def generate_atom_map(topology,
                       new_res,
                       system,
                       positions,
                       system_generator
                       ):
    
    # Create the point mutation engine
    from perses.rjmc.topology_proposal import PointMutationEngine
    point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                                system_generator=system_generator,
                                                chain_id='A', # Denote the chain id allowed to mutate (it's always a string variable)
                                                max_point_mutants=1,
                                                residues_allowed_to_mutate=['2'], # The residue ids allowed to mutate
                                                allowed_mutations=[('2', new_res)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                aggregate=True) # Always allow aggregation
    
    # Create topology proposal
    topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology, extra_sidechain_map=None, demap_CBs=False)
    old_res = topology_proposal.old_residue_name

    # Render atom map
    atom_map_filename = f'atom_map_{old_res}_to_{new_res}.png'
    render_protein_residue_atom_mapping(topology_proposal, atom_map_filename)


In [24]:
amino_acids = ['ALA', 'ARG', 'ASH', 'ASN', 'ASP', 'CYS', 'GLN', 'GLH', 'GLU', 'GLY', 
               'HID', 'HIE', 'HIP', 'HIS', 'ILE', 'LEU', 'LYN',  'LYS',  'MET', 'PHE',
                    'SER', 'THR', 'TRP', 'TYR', 'VAL']
topology, positions, system, system_generator = generate_old_top_pos_sys("asn")
amino_acids.remove('ASN')

for amino_acid in amino_acids:
    generate_atom_map(topology, amino_acid, system, positions, system_generator)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO: